
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## HyCReeW-Q : Overtopping estimation of historical nearshore storms at 3 locations

inputs required: 
  * Historical DWTs
  * MU - TAU intradaily hidrographs time series
  * Historical water levels

in this notebook:
  * Transform storms into hourly time series
  * HyCReWW-Q overtopping estimation of historical events
  

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from datetime import datetime

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.rbf import RBF_Interpolation
from teslakit.mda import Normalize

from teslakit.util.time_operations import npdt64todatetime, fast_reindex_hourly, repair_times_hourly, add_max_storms_mask
   



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/albacid/Projects/TeslaKit_projects'
db = Database(p_data)

# set site
db.SetSite('KWAJALEIN')


# --------------------------------------
# N, S, E profiles
pN = 28 #(36,22)
pS = 8  #(13,1)
pE = 16 #(21,14)
#pN = 35 #28 #(36,22)
#pS = 4  #8  #(13,1)
#pE = 20 #16 #(21,14)


# --------------------------------------
# Load historical TWL
hist_TWL = xr.open_dataset(op.join(p_data, 'sites/KWAJALEIN/TIDE/hist_TWL.nc'))


# --------------------------------------
# load Hycreww RBF coefficients and sim. variables min. and max.

# reef characteristics (AlbaR)
xds = xr.open_dataset(op.join(p_data, 'inputs_Kwajalein','datos_perfiles_albaR','data_profiles.nc'))
xds = xds.drop('zBeach')
reef_N = xds.sel(profile = pN )
reef_S = xds.sel(profile = pS)
reef_E = xds.sel(profile = pE)


# Zbeach characteristics (Sara)
xds_Zbeach = xr.open_dataset(op.join(p_data, 'inputs_Kwajalein','datos_perfiles_sara','zBeach_clean_P1.nc'))
#xds_Zbeach = xr.open_dataset(op.join(p_data, 'inputs_Kwajalein','datos_perfiles_sara','zBeach_perimeter1.nc'))
xds_Zbeach = xds_Zbeach.drop({'xcoor', 'ycoor'})

Zbeach_N = xds_Zbeach.sel(profile = pN )
Zbeach_S = xds_Zbeach.sel(profile = pS)
Zbeach_E = xds_Zbeach.sel(profile = pE)
print(Zbeach_N.zBeach)
print(Zbeach_S.zBeach)
print(Zbeach_E.zBeach)
sys.exit()

# Hycreww limits and coefficients
var_lims, rbf_coeffs = db.Load_HYCREWW_Q()


# --------------------------------------
# MU - TAU intradaily hidrographs time series
xds_mutau_N = xr.open_dataset(op.join(p_data, 'sites', 'KWAJALEIN', 'ESTELA', 'hydrographs', 'MuTau_N.nc'))
xds_mutau_S = xr.open_dataset(op.join(p_data, 'sites', 'KWAJALEIN', 'ESTELA', 'hydrographs', 'MuTau_S.nc'))
xds_mutau_E = xr.open_dataset(op.join(p_data, 'sites', 'KWAJALEIN', 'ESTELA', 'hydrographs', 'MuTau_E.nc'))

#xds_mutau_N = xds_mutau_N.drop('MU')
#xds_mutau_S = xds_mutau_S.drop('MU')
#xds_mutau_E = xds_mutau_E.drop('MU')
print(xds_mutau_N)

#xds_mutau_N = xds_mutau_N.rename({'MU2':'MU'})
#xds_mutau_S = xds_mutau_S.rename({'MU2':'MU'})
#xds_mutau_E = xds_mutau_E.rename({'MU2':'MU'})
#print(xds_mutau_N)


# --------------------------------------
# data for plotting comparison
WAVES_N_h = xr.open_dataset(op.join(p_data, 'sites/KWAJALEIN/WAVES/swells_profiles_N.nc')) # hourly
WAVES_S_h = xr.open_dataset(op.join(p_data, 'sites/KWAJALEIN/WAVES/swells_profiles_S.nc')) # 3-hourly
WAVES_E_h = xr.open_dataset(op.join(p_data, 'sites/KWAJALEIN/WAVES/swells_profiles_E.nc')) # 3-hourly


# --------------------------------------
# data for plotting comparison
WT = xr.open_dataset('/Users/albacid/Projects/TeslaKit_projects/sites/KWAJALEIN/ESTELA/pred_SLP/kma.nc')
print()
print(WT)


<xarray.DataArray 'zBeach' ()>
array(1.473809)
Coordinates:
    profile  int64 28
<xarray.DataArray 'zBeach' ()>
array(1.533132)
Coordinates:
    profile  int64 8
<xarray.DataArray 'zBeach' ()>
array(1.182817)
Coordinates:
    profile  int64 16


SystemExit: 

# Storms to hourly data

In [ ]:
def Intradaily_Hydrograph(xds_mutau):
    '''
    Calculates intradaily hydrograph (hourly) from a time series of storms.
    #storms waves data (hs, tp, dir) and (mu, tau, ss) is needed.

    xds_wvs (waves aggregated):
        xarray.Dataset (time,), Hs, Tp, Dir

    xds_tcs (TCs):
        xarray.Dataset (time,), mu, tau, ss

    returns xarray.Dataset (time,), Hs, Tp, Dir, SS  (hourly)
    '''

    # input data (storms aggregated waves)
    Hs = xds_mutau.Hs.values[:]
    Tp = xds_mutau.Tp.values[:]
    Dir = xds_mutau.Dir.values[:]
    ts = xds_mutau.time.values[:]    
    
    
    # TODO: this should not be needed
    if isinstance(ts[0], np.datetime64):
        ts = [npdt64todatetime(x) for x in ts]
        

    # input data (storms TCs)
    tau = xds_mutau.TAU.values[:]  # storm max. instant (0-1)
    mu = xds_mutau.MU.values[:]
    #ss = xds_tcs.ss.values[:]
    
        
    # storm durations
    s_dur_h = np.array([x.total_seconds()/3600 for x in np.diff(ts)])  # hours    
    s_cs_h = np.cumsum(s_dur_h)  # hours since time start
    s_cs_h = np.insert(s_cs_h,0,0)    

    
    # storm tau max (hourly)
    tau_h = np.floor(s_cs_h[:-1] + s_dur_h * tau[:-1])
    
    
    # aux function
    def CalcHydro(vv, vt, tt, mt):
        '''
        Calculate variable hourly hydrograph.
        vv - var value at max.
        vt - var time (hours since start, at hydrograph extremes)
        tt - tau max time (hours since start).
        mt - mu value
        '''
        
        # var value at hydrographs extremes
        vv_extr = vv * np.power(2*mt-1, 2)

        
        # make it continuous
        vv_extr_cont = (np.roll(vv_extr,1) + vv_extr) / 2
        vv_extr_cont[0] = vv_extr_cont[1]
        vv_extr_cont[-1] = vv_extr_cont[-2]
        
        
        # join hydrograph max. and extremes variable data
        vt_full = np.concatenate([vt, tt])  # concatenate times (used for sorting)
        vv_full = np.concatenate([vv_extr_cont, vv])        

        
        # sort data
        ix = np.argsort(vt_full)
        vt_sf = vt_full[ix]
        vv_sf = vv_full[ix]
        
            
        # interpolate to fill all hours
        h_times = np.arange(vt_sf[0], vt_sf[-1] + 1, 1)
        
        h_values = np.interp(h_times, vt_sf, vv_sf)

        # fix times
        h_times = h_times.astype(int)
        

        return h_values, h_times
    

    # hydrograph variables: hs and ss
    hourly_Hs, hourly_times = CalcHydro(Hs, s_cs_h, tau_h, mu)        
    #hourly_ss, _ = CalcHydro(ss, s_cs_h, tau_h, mu)
    
    
    # resample waves data to hourly (pad Tp and Dir)
    xds_wvs_h = fast_reindex_hourly(xds_mutau)
            
    # select wave variables
    xds_wvs_h = xds_wvs_h[['Hs','Tp','Dir']]

    # add Hs and SS 
    xds_wvs_h['Hs'] =(('time',), hourly_Hs)
    #xds_wvs_h['SS'] =(('time',), hourly_ss)

    return xds_wvs_h



In [ ]:

# calculate intradaily hourly hydrographs for simulated storms
WAVES_N = Intradaily_Hydrograph(xds_mutau_N)
WAVES_S = Intradaily_Hydrograph(xds_mutau_S)
WAVES_E = Intradaily_Hydrograph(xds_mutau_E)
print(WAVES_N)
print()

# repair times: remove duplicates (if any)
WAVES_N = repair_times_hourly(WAVES_N)
WAVES_S = repair_times_hourly(WAVES_S)
WAVES_E = repair_times_hourly(WAVES_E)
print(WAVES_N)
print()

# add mask for max_storms times
#WVS_h = add_max_storms_mask(WVS_h, xds_mutau_N.Runup_max.values, name_mask='max_storms')
#print(WVS_h)
#print()

   

### Plot Hs

In [ ]:
# plot all profiles (1 year zoom)

WAVES_N_h = WAVES_N_h.sel(profile=pN)
WAVES_S_h = WAVES_S_h.sel(profile=pS)
WAVES_E_h = WAVES_E_h.sel(profile=pE)


# plot N
plt.figure(figsize=(20,10))
plt.plot(WAVES_N_h.time, WAVES_N_h.hs, label='Hs original waves')
plt.plot(WAVES_N.time, WAVES_N.Hs,color='C1',label='Hs h from storms')
plt.plot(xds_mutau_N.time_runup_max, xds_mutau_N.Hs, '*r',label='Hs storms')
plt.legend()
plt.xlim(datetime(1997,1,1),datetime(1998,1,1))
plt.title('North profile. pN = ' + str(pN))


# plot S
plt.figure(figsize=(20,10))
plt.plot(WAVES_S_h.time, WAVES_S_h.hs, label='Hs original waves')
plt.plot(WAVES_S.time, WAVES_S.Hs, color='C1',label='Hs h from storms')
plt.plot(xds_mutau_S.time_runup_max, xds_mutau_S.Hs, '*r', label='Hs storms')
plt.legend()
plt.xlim(datetime(1997,1,1),datetime(1998,1,1))
plt.title('South profile. pS = ' + str(pS))


# plot E
plt.figure(figsize=(20,10))
plt.plot(WAVES_E_h.time, WAVES_E_h.hs, label='Hs original waves')
plt.plot(WAVES_E.time, WAVES_E.Hs, color='C1',label='Hs h from storms')
plt.plot(xds_mutau_E.time_runup_max, xds_mutau_E.Hs, '*r', label='Hs storms')
plt.legend()
plt.xlim(datetime(1997,1,1),datetime(1998,1,1))
plt.title('East profile. pE = ' + str(pE))



In [ ]:
# plot South profile & WTs (1 month zoom)

plt.figure(figsize=(20,10))
plt.plot(WAVES_S_h.time, WAVES_S_h.hs, label='Hs original waves')
plt.plot(WAVES_S.time, WAVES_S.Hs, label='Hs from storms')
plt.plot(xds_mutau_S.time_runup_max, xds_mutau_S.Hs, '*r', label='Hs (TWLmax) hydrographs')
plt.legend()
plt.grid()
plt.xlim(datetime(1997,9,1),datetime(1997,9,30))
plt.title('South profile. pS = ' + str(pS))

plt.figure(figsize=(20,10))
plt.plot(WT.time, WT.sorted_bmus,'.', label='WT original')
plt.plot(xds_mutau_S.time, xds_mutau_S.WT-0.3,'ok', label='WT hydrographs ini')
plt.plot(xds_mutau_S.time_runup_max, xds_mutau_S.WT-0.55,'or', label='WT hydrographs max')
plt.xlim(datetime(1997,9,1),datetime(1997,9,30))
plt.legend()
plt.grid()



### plot Tp

In [ ]:

# plot S (1 yr)
plt.figure(figsize=(20,10))
plt.plot(WAVES_S_h.time, WAVES_S_h.tp, label='Tp original waves')
plt.plot(WAVES_S.time, WAVES_S.Tp, color='C1',label='Tp h from storms')
plt.legend()
plt.xlim(datetime(1997,1,1),datetime(1998,1,1))
plt.title('South profile. pS = ' + str(pS))


# plot South profile & WTs (1 month zoom)

plt.figure(figsize=(20,10))
plt.plot(WAVES_S_h.time, WAVES_S_h.tp, label='Tp original waves')
plt.plot(WAVES_S.time, WAVES_S.Tp, label='Tp from storms')
plt.legend()
plt.grid()
plt.xlim(datetime(1997,9,1),datetime(1997,9,30))
plt.title('South profile. pS = ' + str(pS))


### plot scatter (hourly)

In [ ]:
print(WAVES_S_h) # 3-h
print(WAVES_S) # 1-h
print()

WAVES_S_h = WAVES_S_h.resample(time='1H').interpolate('linear')
WAVES_S_h = WAVES_S_h.sel(time=slice(WAVES_S.time[0],WAVES_S.time[-1]))
print(WAVES_S_h) # 1-h
print(WAVES_S) # 1-h
print()

var1 = 'hs'
var2 = 'Hs'
plt.figure(figsize=(10,10))
plt.plot(WAVES_S_h[var1], WAVES_S[var2], '.')
plt.xlabel(var2 + ' original waves')
plt.ylabel(var2 + ' from storms')
plt.xlim(-0.5, 4)
plt.ylim(-0.5, 4)
plt.title('hourly ' + var2)

var1 = 'tp'
var2 = 'Tp'
plt.figure(figsize=(10,10))
plt.plot(WAVES_S_h[var1], WAVES_S[var2], '.')
plt.xlabel(var2 + ' original waves')
plt.ylabel(var2 + ' from storms')
plt.xlim(0, 22)
plt.ylim(0, 22)
plt.title('hourly ' + var2)

sys.exit()

### plot scatter (3-hourly)

In [ ]:
print(WAVES_S_h) # 3-h
print(WAVES_S) # 1-h
print()

WAVES_S_h = WAVES_S_h.sel(time=slice(WAVES_S.time[0],WAVES_S.time[-1]))
WAVES_S = WAVES_S.sel(time=WAVES_S_h.time)
print(WAVES_S_h) # 3-h
print(WAVES_S) # 3-h
print()

var1 = 'hs'
var2 = 'Hs'
plt.figure(figsize=(10,10))
plt.plot(WAVES_S_h[var1], WAVES_S[var2], '.')
plt.xlabel(var2 + ' original waves')
plt.ylabel(var2 + ' from storms')
plt.xlim(-0.5, 4)
plt.ylim(-0.5, 4)
plt.title('3-hourly ' + var2)

var1 = 'tp'
var2 = 'Tp'
plt.figure(figsize=(10,10))
plt.plot(WAVES_S_h[var1], WAVES_S[var2], '.')
plt.xlabel(var2 + ' original waves')
plt.ylabel(var2 + ' from storms')
plt.xlim(0, 22)
plt.ylim(0, 22)
plt.title('3-hourly ' + var2)


sys.exit()


## Hycreww RBF Interpolation

##### Set parameters for Hycreww: TWL, Reef Slope, Beach Slope, Reef Width, Zbeach, Cf

In [ ]:

dset_N = xr.Dataset(
            {   'hs':(('time'), WAVES_N.Hs.values),
                'tp':(('time'), WAVES_N.Tp.values),
                'bslope':(('time'), np.ones(len(WAVES_N.time),)*reef_N['Beach Slope'].values),
                'rslope':(('time'), np.ones(len(WAVES_N.time),)*reef_N['Fore Slope'].values),
                'rwidth':(('time'), np.ones(len(WAVES_N.time),)*reef_N['Wreef'].values),
                'Zb':(('time'), np.ones(len(WAVES_N.time),)*Zbeach_N['zBeach'].values),
                'cf':(('time'), np.ones(len(WAVES_N.time),)*0.1),
            },coords = {'time': WAVES_N.time.values})

dset_S = xr.Dataset(
            {   'hs':(('time'), WAVES_S.Hs.values),
                'tp':(('time'), WAVES_S.Tp.values),
                'bslope':(('time'), np.ones(len(WAVES_S.time),)*reef_S['Beach Slope'].values),
                'rslope':(('time'), np.ones(len(WAVES_S.time),)*reef_S['Fore Slope'].values),
                'rwidth':(('time'), np.ones(len(WAVES_S.time),)*reef_S['Wreef'].values),
                'Zb':(('time'), np.ones(len(WAVES_S.time),)*Zbeach_S['zBeach'].values),
                'cf':(('time'), np.ones(len(WAVES_S.time),)*0.1),
            },coords = {'time': WAVES_S.time.values})

dset_E = xr.Dataset(
            {   'hs':(('time'), WAVES_E.Hs.values),
                'tp':(('time'), WAVES_E.Tp.values),
                'bslope':(('time'), np.ones(len(WAVES_E.time),)*reef_E['Beach Slope'].values),
                'rslope':(('time'), np.ones(len(WAVES_E.time),)*reef_E['Fore Slope'].values),
                'rwidth':(('time'), np.ones(len(WAVES_E.time),)*reef_E['Wreef'].values),
                'Zb':(('time'), np.ones(len(WAVES_E.time),)*Zbeach_E['zBeach'].values),
                'cf':(('time'), np.ones(len(WAVES_E.time),)*0.1),
            },coords = {'time': WAVES_E.time.values})

dset_N = dset_N.sel(time=slice(dset_N.time.values[0],hist_TWL.time.values[-1]))
dset_S = dset_S.sel(time=slice(dset_N.time.values[0],hist_TWL.time.values[-1]))
dset_E = dset_E.sel(time=slice(dset_N.time.values[0],hist_TWL.time.values[-1]))

dset_N['level'] = hist_TWL.TWL.sel(time=dset_N.time)
dset_S['level'] = hist_TWL.TWL.sel(time=dset_S.time)
dset_E['level'] = hist_TWL.TWL.sel(time=dset_E.time)

dset_N['hs_lo2'] = (dset_N.hs/(1.5613*dset_N.tp**2))
dset_S['hs_lo2'] = (dset_S.hs/(1.5613*dset_S.tp**2))
dset_E['hs_lo2'] = (dset_E.hs/(1.5613*dset_E.tp**2))

print(dset_N)
print(dset_S)
print(dset_E)

In [ ]:
# Adjust variables to fit within Hycreww limits

for var in var_lims.keys():

    dset_N[var] =  dset_N[var].where(dset_N[var]>=var_lims[var][0], var_lims[var][0] )
    dset_N[var] =  dset_N[var].where(dset_N[var]<=var_lims[var][1], var_lims[var][1] )    
    
    dset_S[var] =  dset_S[var].where(dset_S[var]>=var_lims[var][0], var_lims[var][0] )
    dset_S[var] =  dset_S[var].where(dset_S[var]<=var_lims[var][1], var_lims[var][1] )    
    
    dset_E[var] =  dset_E[var].where(dset_E[var]>=var_lims[var][0], var_lims[var][0] )
    dset_E[var] =  dset_E[var].where(dset_E[var]<=var_lims[var][1], var_lims[var][1] )    
    
print(dset_N)
print(dset_S)
print(dset_E)


### HyCReWW-Q

In [ ]:
# HyCReWW Function

def hycreww_Q(var_lims, rbf_coeffs, dset):
    '''
    Calculates RunUp using hycreww RBFs (level) and linear interpolation (Runup)
    
    var_lims   - hycreww variables min and max limits
    rbf_coeffs - hycreww rbf coefficients
    dset       - input dataset (WL, Hs, Tp, Reef Slope, Beach Slope, Reef Width, Zbeach, Cf )
    
    '''
    # RBF wave conditions 
    rbf_hs = [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]
    rbf_hs_lo = [0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05, 0.005, 0.025, 0.05 ]
    rbf_vns = ['level', 'rslope', 'bslope', 'rwidth', 'Zb', 'cf']

    # RBF parameters
    ix_sc = [0, 1, 2, 3, 4, 5]
    ix_dr = []
    minis = [var_lims[x][0] for x in rbf_vns]
    maxis = [var_lims[x][1] for x in rbf_vns]


    # discard data outside limits
#     for vn in var_lims.keys():
#         dset = dset.isel(num=np.where((dset[vn] >= var_lims[vn][0]) & (dset[vn] <= var_lims[vn][1]))[0])


    # RBF dataset to interpolate
    ds_in = dset[rbf_vns]
    ds_in=([dset.level.values],[dset.rslope.values],[dset.bslope.values],[dset.rwidth.values],[dset.Zb.values],[dset.cf.values])
    ds_in=np.transpose(ds_in)[:,0,:]

    # normalize data
    ds_nm ,_ ,_ = Normalize(ds_in, ix_sc, ix_dr, minis=minis, maxis=maxis)

    # RBF interpolation (with all cases?)
    ru_out = []
    for rc in rbf_coeffs:
        ro = RBF_Interpolation(rc['constant'], rc['coeff'], rc['nodes'], ds_nm.T)
        ru_out.append(ro)
    ru_z = np.array(ru_out)

    # Q Linear interpolation (hs, hs_lo -> runup)
    Q = []
    for c  in range(len(dset.time)):
        vq = griddata((rbf_hs, rbf_hs_lo), ru_z[:,c], (dset.hs.values[c], dset.hs_lo2.values[c]), method='linear')
        Q.append(vq)
    Q = np.array(Q)

    # store runup alongside input data
    dset_out = dset.copy()  
    dset_out['Q'] = (('time'),Q)

    return dset_out

In [ ]:
out_sim_N = hycreww_Q(var_lims, rbf_coeffs, dset_N)
print(out_sim_N)

out_sim_S = hycreww_Q(var_lims, rbf_coeffs, dset_S)
print(out_sim_S)

out_sim_E = hycreww_Q(var_lims, rbf_coeffs, dset_E)
print(out_sim_E)


### We undo the box-cox transformation

In [ ]:
from scipy.special import boxcox, inv_boxcox

# inv_boxcox(y, lmbda) 
# y = (x**lmbda - 1) / lmbda  if lmbda != 0
#     log(x)                  if lmbda == 0
    
out_sim_N['Qinv']=inv_boxcox(out_sim_N.Q.values, 0.2)
out_sim_S['Qinv']=inv_boxcox(out_sim_S.Q.values, 0.2)
out_sim_E['Qinv']=inv_boxcox(out_sim_E.Q.values, 0.2)

In [ ]:
print(out_sim_E)

In [ ]:
# Save
out_sim_N.to_netcdf(op.join(p_data, 'sites/KWAJALEIN/HYCREWW/hist_Q_storms_N' + str(pN) + '.nc'))
out_sim_S.to_netcdf(op.join(p_data, 'sites/KWAJALEIN/HYCREWW/hist_Q_storms_S' + str(pS) + '.nc'))
out_sim_E.to_netcdf(op.join(p_data, 'sites/KWAJALEIN/HYCREWW/hist_Q_storms_E' + str(pE) + '.nc'))


### Plot

In [ ]:

out_sim_N = xr.open_dataset(op.join(p_data, 'sites/KWAJALEIN/HYCREWW/hist_Q_storms_N' + str(pN) + '.nc'))
out_sim_S = xr.open_dataset(op.join(p_data, 'sites/KWAJALEIN/HYCREWW/hist_Q_storms_S' + str(pS) + '.nc'))
out_sim_E = xr.open_dataset(op.join(p_data, 'sites/KWAJALEIN/HYCREWW/hist_Q_storms_E' + str(pE) + '.nc'))

print(out_sim_N)
print(out_sim_S)
print(out_sim_E)

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(20,10))

axs[0].plot(out_sim_N.time, out_sim_N.Qinv, color='g', label='Q north')
axs[1].plot(out_sim_S.time, out_sim_S.Qinv, color='b', label='Q South')
axs[2].plot(out_sim_E.time, out_sim_E.Qinv, color='r', label='Q East')
axs[0].legend()
axs[1].legend()
axs[2].legend()
plt.suptitle('Overtopping (l/s/m)')



fig, axs = plt.subplots(3,1, figsize=(20,10))

axs[0].plot(out_sim_N.time, out_sim_N.Q, label='Q north')
axs[1].plot(out_sim_S.time, out_sim_S.Q, label='Q South')
axs[2].plot(out_sim_E.time, out_sim_E.Q, label='Q East')

plt.suptitle('Overtopping (l/s/m)')

In [ ]:
plt.figure(figsize=(20,10))

plt.plot(out_sim_S.time, out_sim_S.Qinv, color='b', label='Q South')
plt.legend()
plt.legend()
plt.title('Overtopping (l/s/m)')


plt.figure(figsize=(20,10))

plt.plot(out_sim_S.time.values[0:24*30], out_sim_S.Qinv.values[0:24*30], color='b', label='Q South')
plt.legend()
plt.legend()
plt.title('Overtopping (l/s/m)')


# histogram
plt.figure(figsize=(20,10))
bins_number = 40 
bins = np.linspace(np.nanmin(out_sim_S.Qinv.values), np.nanmax(out_sim_S.Qinv.values), bins_number)

n, _, _ = plt.hist(out_sim_S.Qinv.values, bins)
plt.show()


# percentiles
q = np.linspace(0,100,41)
perc = np.nanpercentile(out_sim_S.Qinv.values, q)

plt.figure(figsize=(20,10))
plt.plot(perc, q)
plt.plot(perc, q, '.')
plt.xlabel('Q')
plt.ylabel('%')
plt.grid()

print(q)
print()
print(perc)